This simple notebook is an update of the notebook by Jake Vanderplas at https://jakevdp.github.io/blog/2013/06/15/numba-vs-cython-take-2/ to run on Python 3.5. It compares usage of cython and numba illustrating the ability to vastly increase speed of computation with some simple tricks. 


A very pithy description of this is that `X` contains $1000$ points in 3D space. The Euclidian distance between any two points can be calculated (vector length). This can be represented by a 1000 $\times$ 1000 array containing all possible pairs. Calculating this in Python is slow. This can be easily sped up by "precompiling" the code. 

For more details on how to migrate to compiled code see [Hans Lantangen's tutorial](http://hplgit.github.io/primer.html/doc/pub/cython/cython-solarized.html).

In [21]:
# We are going to load the Cython extension for Jupyter. This allows us to 
# put a few things into Python code. Cython then turns it into C code, 
# compiles it, and runs it in the background. 

%load_ext Cython

import numpy as np
X = np.random.random((1000, 3))
D = np.empty((1000, 1000))

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [22]:
# Pure python version

def pairwise_python(X, D):
    M = X.shape[0]
    N = X.shape[1]
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

In [24]:
# numba version

import numpy as np
from numba import double
from numba.decorators import jit

# The next line is the only "change to the code"
@jit     # (arg_types=[double[:,:], double[:,:]])
def pairwise_numba(X, D):
    M = X.shape[0]
    N = X.shape[1]
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

In [25]:
%%cython
# The above declares that this cell is Cython code.

# Load some cheats that help cython
cimport cython

# Load the sqrt function, but with cimport, also import knowledge of the variable 
# type used (it's a floating point)
from libc.math cimport sqrt

# Tell cython not to check if my indices called ae actually inside the array
# This is a good move if you are ABSOLUTELY sure of it. It's a bad move otherwise. 
@cython.boundscheck(False)

# This disallows use of negative indices. 
@cython.wraparound(False)
def pairwise_cython(double[:, ::1] X, double[:, ::1] D):
    cdef int M = X.shape[0]
    cdef int N = X.shape[1]
    cdef double tmp, d
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = sqrt(d)
    return D

In [26]:
%timeit pairwise_python(X, D)

3.83 s ± 139 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit pairwise_numba(X, D)

2.72 ms ± 92.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%timeit pairwise_cython(X, D)

2.82 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
